# 1. 导包

In [2]:
import time, math, os
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
from collections import defaultdict
import collections
warnings.filterwarnings('ignore')

In [3]:
data_path = './data_raw/'
save_path = './tmp_results/'

In [4]:
# 节约内存的一个标配函数
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,100*(start_mem-end_mem)/start_mem,(time.time()-starttime)/60))
    return df

# 2. 读取采样或全量数据

In [5]:
# debug模式：从训练集中划出一部分数据来调试代码
def get_all_click_sample(data_path, sample_nums=10000):
    """
        训练集中采样一部分数据调试
        data_path: 原数据的存储路径
        sample_nums: 采样数目（这里由于机器的内存限制，可以采样用户做）
    """
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    all_user_ids = all_click.user_id.unique() # 去除重复后，从大到小排列

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) 
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

# 读取点击数据，这里分成线上和线下，如果是为了获取线上提交结果，应该将测试集中的点击数据合并到总的数据中
# 如果是为了线下验证模型的有效性或者特征的有效性，可以只使用训练集
def get_all_click_df(data_path='./data_raw/', offline=True):
    if offline:
        all_click = pd.read_csv(data_path + 'train_click_log.csv')
    else:
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')

        all_click = trn_click.append(tst_click)
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

## 随机取数：numpy.random.choice(a, size=None, replace=True, p=None)
- 从a(只要是ndarray都可以，但必须是一维的)中随机抽取数字，并组成指定大小(size)的数组
- replace:True表示可以取相同数字，False表示不可以取相同数字
- 数组p：与数组a相对应，表示取数组a中每个元素的概率，默认为选取每个元素的概率相同

参考：https://blog.csdn.net/ImwaterP/article/details/96282230

## Pandas数据去重：DataFrame.drop_duplicates(subset=None, keep='first', inplace=False)
- subset : column label or sequence of labels, optional 用来指定特定的列，默认所有列
- keep : {‘first’, ‘last’, False}, default ‘first’ 删除重复项并保留第一次出现的项
- inplace : boolean, default False 是直接在原来数据上修改还是保留一个副本

参考：https://laoai.blog.csdn.net/article/details/83926840

In [6]:
# 全量训练集
all_click_df = get_all_click_df(offline=False)

In [7]:
all_click_df 

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5
...,...,...,...,...,...,...,...,...,...
518005,221924,70758,1508211323220,4,3,2,1,25,2
518006,207823,331116,1508211542618,4,3,2,1,25,1
518007,207823,234481,1508211850103,4,3,2,1,25,1
518008,207823,211442,1508212189949,4,3,2,1,25,1


# 3. 获取字典：用户-文章-点击时间

In [8]:
# 根据点击时间获取用户的点击文章序列   {user1: {item1: time1, item2: time2..}...}
def get_user_item_time(click_df):
    
    click_df = click_df.sort_values('click_timestamp')
    
    def make_item_time_pair(df):
        """
         字典：{(文章ID，时间戳)}
        """
        return list(zip(df['click_article_id'], df['click_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'}) #将第0列或'o'列的名称改为'.'
    
    """
    user_item_time_dict:
    把dataframe转为字典：{用户ID,[(文章ID1，时间戳),(文章ID2，时间戳),...]}
    """
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

## Pandas排序函数：DataFrame.sort_values(by=‘##’,axis=0,ascending=True, inplace=False, na_position=‘last’)
将数据集依照某个字段中的数据进行排序(可指定根据列/行排序)
- by	指定列名(axis=0或’index’)或索引值(axis=1或’columns’)
- axis	若axis=0或’index’，则按照指定列中数据大小排序；若axis=1或’columns’，则按照指定索引中数据大小排序，默认axis=0
- ascending	是否按指定列的数组升序排列，默认为True，即升序排列
- inplace	是否用排序后的数据集替换原来的数据，默认为False，即不替换
- na_position	{‘first’,‘last’}，设定缺失值的显示位置

参考：https://blog.csdn.net/MsSpark/article/details/83154128

## Pandas分组函数：
DataFrame.groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=<object object>, observed=False, dropna=True)

参考：https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html
    https://www.cnblogs.com/Yanjy-OnlyOne/p/11217802.html
    https://zhuanlan.zhihu.com/p/101284491

## Pandas重置索引:DataFrame.reset_index()
数据清洗时，会将带空值的行删除，此时DataFrame或Series类型的数据不再是连续的索引，可以使用reset_index()重置索引

# 4. 获取点击最多的topk个文章

In [9]:
# 获取近期点击最多的文章
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

## Pandas确认数据出现的频率：DataFrame.value_counts()
需要指定列或行
参考：https://www.jianshu.com/p/f773b4b82c66

# 5. itemcf的物品相似度计算
## 基于物品的协同过滤算法

参考[1]：https://blog.csdn.net/wickedvalley/article/details/79927699

参考[2]：https://www.jianshu.com/p/16dbcf8a2f24

In [10]:
def itemcf_sim(df):
    """
        文章与文章之间的相似性矩阵计算
        param df: 数据表
        item_created_time_dict:  文章创建时间的字典
        return : 文章与文章的相似性矩阵
        思路: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习)， 在多路召回部分会加上关联规则的召回策略
    """
    
    user_item_time_dict = get_user_item_time(df)
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int) #当key不存在时，返回默认值为0 https://www.jianshu.com/p/bbd258f99fd3
    for user, item_time_list in tqdm(user_item_time_dict.items()): #tqdm-进度条库：https://www.jianshu.com/p/21cf48be6bf6
        # 在基于商品的协同过滤优化的时候可以考虑时间因素
        for i, i_click_time in item_time_list:
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for j, j_click_time in item_time_list:
                if(i == j):
                    continue
                i2i_sim[i].setdefault(j, 0) #dict.setdefault(key, default=None) 
                                                            #key -- 查找的键值
                                                            #default -- 键不存在时，设置的默认键值
                
                i2i_sim[i][j] += 1 / math.log(len(item_time_list) + 1)
                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items(): #{用户ID：{文章ID：Wij}}
        for j, wij in related_items.items(): #{文章ID：Wij}
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j]) #余弦相似度
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim.pkl', 'wb'))
    
    return i2i_sim_

In [11]:
i2i_sim = itemcf_sim(all_click_df)
# i2i_sim = pickle.load(itemcf_i2i_sim.pkl)

100%|██████████| 250000/250000 [00:21<00:00, 11424.25it/s]


## 推荐系统计算物品间相似度
参考：https://blog.csdn.net/eyeofeagle/article/details/83213748

### 使用余弦定理计算
参考：https://zhuanlan.zhihu.com/p/154108167
### 使用欧氏距离计算
参考：https://zhuanlan.zhihu.com/p/155960197
## dictionary.setdefault(keyname, value)
使用指定的键返回项目的值，如果键不存在，则插入这个具有指定值的键

参考：https://www.w3school.com.cn/python/ref_dictionary_setdefault.asp

# 6. itemcf的文章推荐

In [12]:
# 基于商品的召回i2i
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列   {user1: {item1: time1, item2: time2..}...}
        :param i2i_sim: 字典，文章相似性矩阵
        :param sim_item_topk: 整数， 选择与当前文章最相似的前k篇文章
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，用户召回补全        
        return: 召回的文章列表 {item1:score1, item2: score2...}
        注意: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习)， 在多路召回部分会加上关联规则的召回策略
    """
    
    # 获取用户历史交互的文章
    # user_item_time_dict:{用户ID,[(文章ID1，时间戳),(文章ID2，时间戳),...]}

    user_hist_items = user_item_time_dict[user_id]
    
    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
        # 按时间降序排序，取到前topk
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items:
                continue
                
            item_rank.setdefault(j, 0)
            item_rank[j] +=  wij
    
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
        
    return item_rank

## Python sorted()函数：对所有可迭代的对象进行排序操作

### sort与sorted区别：
- sort是应用在list上的方法，sorted可以对所有可迭代的对象进行排序操作
- list的sort方法返回的是对已经存在的列表进行操作，无返回值
- 而内建函数sorted方法返回的是一个新的list，而不是在原来的基础上进行的操作
### sorted(iterable, cmp=None, key=None, reverse=False)
- iterable -- 可迭代对象
- cmp -- 比较的函数，这个具有两个参数，参数的值都是从可迭代对象中取出，此函数必须遵守的规则为，大于则返回1，小于则返回-1，等于则返回0
- key -- 主要是用来进行比较的元素，只有一个参数，具体的函数的参数就是取自于可迭代对象中，指定可迭代对象中的一个元素来进行排序
- reverse -- 排序规则，reverse = True 降序 ， reverse = False 升序（默认）
- 返回重新排序的列表

参考：https://www.runoob.com/python/python-func-sorted.html

# 7. 给每个用户根据物品的协同过滤推荐文章

In [ ]:
# 定义
user_recall_items_dict = collections.defaultdict(dict)

# 获取 用户 - 文章 - 点击时间的字典
user_item_time_dict = get_user_item_time(all_click_df)

# 去取文章相似度
i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))

# 相似文章的数量
sim_item_topk = 10

# 召回文章数量
recall_item_num = 10

# 用户热度补全
item_topk_click = get_item_topk_click(all_click_df, k=50)

for user in tqdm(all_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, 
                                                        sim_item_topk, recall_item_num, item_topk_click)

 33%|███▎      | 83239/250000 [17:15<25:33, 108.77it/s] 

# 8.召回字典转换成df

In [ ]:
# 将字典的形式转换成df
user_item_score_list = []

for user, items in tqdm(user_recall_items_dict.items()):
    for item, score in items:
        user_item_score_list.append([user, item, score])

recall_df = pd.DataFrame(user_item_score_list, columns=['user_id', 'click_article_id', 'pred_score'])

# 9.生成提交文件

In [ ]:
# 生成提交文件
def submit(recall_df, topk=5, model_name=None):
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 判断是不是每个用户都有5篇文章及以上
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk
    
    del recall_df['pred_score']
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    # 按照提交格式定义列名
    submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
                                                  3: 'article_3', 4: 'article_4', 5: 'article_5'})
    
    save_name = save_path + model_name + '_' + datetime.today().strftime('%m-%d') + '.csv'
    submit.to_csv(save_name, index=False, header=True)

In [ ]:
# 获取测试集
tst_click = pd.read_csv(data_path + 'testA_click_log.csv')
tst_users = tst_click['user_id'].unique()

# 从所有的召回数据中将测试集中的用户选出来
tst_recall = recall_df[recall_df['user_id'].isin(tst_users)]

# 生成提交文件
submit(tst_recall, topk=5, model_name='itemcf_baseline')